In [1]:
import datetime
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import xgboost as xgb
import random
import zipfile
import time
import shutil
from sklearn.metrics import log_loss

In [2]:
gender_train = pd.read_csv('gender_age_train.csv')
gender_test = pd.read_csv('gender_age_test.csv')
phone_brand = pd.read_csv('phone_brand_device_model.csv')
app_events = pd.read_csv('app_events.csv', dtype={'app_id': np.str})
app_labels = pd.read_csv('app_labels.csv')
labels = pd.read_csv('label_categories.csv')
events = pd.read_csv('events_new.csv', index_col=0)

In [7]:
app_ev = app_events.drop(['is_installed', 'is_active'],axis=1)

In [10]:
app_cnt = app_ev.groupby('app_id', as_index=False)['event_id'].count()
app_cnt.head()

,app_id,event_id
0,-9221156934682287334,21
1,-9220899153371182692,25
2,-9218487885271516150,2
3,-9218487885267037129,6
4,-9218310540360546691,38


In [34]:
common_apps = app_cnt[app_cnt.event_id>app_cnt.quantile(.99).event_id].app_id.values

In [37]:
app_cnt.quantile(.99).event_id

34061.559999999998

In [36]:
print app_ev.shape
print app_ev[app_ev.app_id.isin(common_apps)].shape

(32473067, 2)
(19213345, 2)


In [38]:
# make it like a event-id - text with the app_ids (e.g. "100 101 10020 ...")
# then use bag of words

MemoryError: 

In [ ]:
data = pd.merge(events, app_events, on='event_id')

In [ ]:
phone_brand.drop_duplicates(subset='device_id',keep='first', inplace=True)

In [ ]:
data.is_morning = data.is_morning.astype(int)
data.is_night = data.is_night.astype(int)
data.is_day = data.is_day.astype(int)
data.is_evening = data.is_evening.astype(int)

In [ ]:
train_data = pd.merge(gender_train, data, on='device_id', how='left')
test_data = pd.merge(gender_test, data, on='device_id', how='left')

In [ ]:
train_data = pd.merge(train_data, phone_brand, on='device_id')
test_data = pd.merge(test_data, phone_brand, on='device_id')

In [ ]:
train_data = train_data.fillna(-1)
test_data = test_data.fillna(-1)

In [ ]:
app_events_sum = app_events.groupby(['event_id'])

In [ ]:
app_cnt = app_events.groupby(['event_id'], as_index=False)['app_id'].count()
installed_sum = app_events.groupby(['event_id'], as_index=False)['is_installed'].sum()
active_sum = app_events.groupby(['event_id'], as_index=False)['is_active'].sum()

In [ ]:
app_data_med = pd.merge(app_cnt, installed_sum, on=['event_id'])
app_data = pd.merge(app_data_med, active_sum, on=['event_id'])

In [ ]:
app_data['active_share'] = app_data.is_active/app_data.app_id

In [ ]:
train_data = train_data.drop(['gender','age'], axis=1)

In [ ]:
#labels = labels.fillna('no_category')

In [ ]:
train_data.head(2)

In [ ]:
test_data.head(2)

In [ ]:
# count devices in events
hour_cnt = train_data.groupby(['device_id'], as_index=False)['hour'].count().rename(columns={'hour':'device_cnt'})
# hour activities
hour_avg = train_data.groupby(['device_id'], as_index=False)['hour'].mean().rename(columns={'hour':'hour_avg'})
hour_med = train_data.groupby(['device_id'], as_index=False)['hour'].median().rename(columns={'hour':'hour_med'})
hour_min = train_data.groupby(['device_id'], as_index=False)['hour'].min().rename(columns={'hour':'hour_min'})
hour_max = train_data.groupby(['device_id'], as_index=False)['hour'].max().rename(columns={'hour':'hour_max'})
# weekday activities
weekday_avg = train_data.groupby(['device_id'], as_index=False)['weekday'].mean().rename(columns={'weekday':'weekday_avg'})
weekday_med = train_data.groupby(['device_id'], as_index=False)['weekday'].median().rename(columns={'weekday':'weekday_med'})
weekday_min = train_data.groupby(['device_id'], as_index=False)['weekday'].min().rename(columns={'weekday':'weekday_min'})
weekday_max = train_data.groupby(['device_id'], as_index=False)['weekday'].max().rename(columns={'weekday':'weekday_max'})
# part of day activities
morning_sum = train_data.groupby(['device_id'], as_index=False)['is_morning'].sum().rename(columns={'is_morning':'morning_sum'})
morning_avg = train_data.groupby(['device_id'], as_index=False)['is_morning'].mean().rename(columns={'is_morning':'morning_avg'})
day_sum = train_data.groupby(['device_id'], as_index=False)['is_day'].sum().rename(columns={'is_morning':'day_sum'})
day_avg = train_data.groupby(['device_id'], as_index=False)['is_day'].mean().rename(columns={'is_morning':'day_avg'})
night_sum = train_data.groupby(['device_id'], as_index=False)['is_night'].sum().rename(columns={'is_night':'night_sum'})
night_avg = train_data.groupby(['device_id'], as_index=False)['is_night'].mean().rename(columns={'is_night':'night_avg'})
evening_sum = train_data.groupby(['device_id'], as_index=False)['is_evening'].sum().rename(columns={'is_evening':'evening_sum'})
evening_avg = train_data.groupby(['device_id'], as_index=False)['is_evening'].mean().rename(columns={'is_evening':'evening_avg'})

In [ ]:
# count devices in events
hour_cnt_test = test_data.groupby(['device_id'], as_index=False)['hour'].count().rename(columns={'hour':'device_cnt'})
# hour activities
hour_avg_test = test_data.groupby(['device_id'], as_index=False)['hour'].mean().rename(columns={'hour':'hour_avg'})
hour_med_test = test_data.groupby(['device_id'], as_index=False)['hour'].median().rename(columns={'hour':'hour_med'})
hour_min_test = test_data.groupby(['device_id'], as_index=False)['hour'].min().rename(columns={'hour':'hour_min'})
hour_max_test = test_data.groupby(['device_id'], as_index=False)['hour'].max().rename(columns={'hour':'hour_max'})
# weekday activities
weekday_avg_test = test_data.groupby(['device_id'], as_index=False)['weekday'].mean().rename(columns={'weekday':'weekday_avg'})
weekday_med_test = test_data.groupby(['device_id'], as_index=False)['weekday'].median().rename(columns={'weekday':'weekday_med'})
weekday_min_test = test_data.groupby(['device_id'], as_index=False)['weekday'].min().rename(columns={'weekday':'weekday_min'})
weekday_max_test = test_data.groupby(['device_id'], as_index=False)['weekday'].max().rename(columns={'weekday':'weekday_max'})
# part of day activities
morning_sum_test = test_data.groupby(['device_id'], as_index=False)['is_morning'].sum().rename(columns={'is_morning':'morning_sum'})
morning_avg_test = test_data.groupby(['device_id'], as_index=False)['is_morning'].mean().rename(columns={'is_morning':'morning_avg'})
day_sum_test = test_data.groupby(['device_id'], as_index=False)['is_day'].sum().rename(columns={'is_morning':'day_sum'})
day_avg_test = test_data.groupby(['device_id'], as_index=False)['is_day'].mean().rename(columns={'is_morning':'day_avg'})
night_sum_test = test_data.groupby(['device_id'], as_index=False)['is_night'].sum().rename(columns={'is_night':'night_sum'})
night_avg_test = test_data.groupby(['device_id'], as_index=False)['is_night'].mean().rename(columns={'is_night':'night_avg'})
evening_sum_test = test_data.groupby(['device_id'], as_index=False)['is_evening'].sum().rename(columns={'is_evening':'evening_sum'})
evening_avg_test = test_data.groupby(['device_id'], as_index=False)['is_evening'].mean().rename(columns={'is_evening':'evening_avg'})

In [ ]:
result_train = pd.merge(hour_cnt, hour_avg, on='device_id')
result_train = pd.merge(result_train, hour_med, on='device_id')
result_train = pd.merge(result_train, hour_min, on='device_id')
result_train = pd.merge(result_train, hour_max, on='device_id')
result_train = pd.merge(result_train, weekday_avg, on='device_id')
result_train = pd.merge(result_train, weekday_med, on='device_id')
result_train = pd.merge(result_train, weekday_min, on='device_id')
result_train = pd.merge(result_train, weekday_max, on='device_id')
result_train = pd.merge(result_train, morning_sum, on='device_id')
result_train = pd.merge(result_train, morning_avg, on='device_id')
result_train = pd.merge(result_train, day_sum, on='device_id')
result_train = pd.merge(result_train, day_avg, on='device_id')
result_train = pd.merge(result_train, evening_sum, on='device_id')
result_train = pd.merge(result_train, evening_avg, on='device_id')
result_train = pd.merge(result_train, night_sum, on='device_id')
result_train = pd.merge(result_train, night_avg, on='device_id')

In [ ]:
result_test = pd.merge(hour_cnt_test, hour_avg_test, on='device_id')
result_test = pd.merge(result_test, hour_med_test, on='device_id')
result_test = pd.merge(result_test, hour_min_test, on='device_id')
result_test = pd.merge(result_test, hour_max_test, on='device_id')
result_test = pd.merge(result_test, weekday_avg_test, on='device_id')
result_test = pd.merge(result_test, weekday_med_test, on='device_id')
result_test = pd.merge(result_test, weekday_min_test, on='device_id')
result_test = pd.merge(result_test, weekday_max_test, on='device_id')
result_test = pd.merge(result_test, morning_sum_test, on='device_id')
result_test = pd.merge(result_test, morning_avg_test, on='device_id')
result_test = pd.merge(result_test, day_sum_test, on='device_id')
result_test = pd.merge(result_test, day_avg_test, on='device_id')
result_test = pd.merge(result_test, evening_sum_test, on='device_id')
result_test = pd.merge(result_test, evening_avg_test, on='device_id')
result_test = pd.merge(result_test, night_sum_test, on='device_id')
result_test = pd.merge(result_test, night_avg_test, on='device_id')

In [ ]:
result_train.head()

In [ ]:
result_test.head()

In [ ]:
def map_column(table, f):
    labels = sorted(table[f].unique())
    mappings = dict()
    for i in range(len(labels)):
        mappings[labels[i]] = i
    table = table.replace({f: mappings})
    return table

In [ ]:
phone_brand['phone+brand'] = phone_brand.phone_brand + phone_brand.device_model
phone_brand = phone_brand.drop(['phone_brand','device_model'],axis=1)
phone_brand = pd.get_dummies(phone_brand)

In [ ]:
result_test = pd.merge(result_test, phone_brand, on='device_id')
result_train = pd.merge(result_train, phone_brand, on='device_id')

In [ ]:
result_train = pd.merge(result_train, gender_train, on='device_id')
result_train = result_train.drop(['gender','age'],axis=1)

In [ ]:
result_train = map_column(result_train, 'group')

In [ ]:
random.seed(2016)

def run_xgb(train, test, features, target, eta=0.1, random_state=0):
    #eta = 0.1
    max_depth = 3
    subsample = 0.7
    colsample_bytree = 0.7
    start_time = time.time()

    print('XGBoost params. ETA: {}, MAX_DEPTH: {}, SUBSAMPLE: {}, COLSAMPLE_BY_TREE: {}'.format(eta, max_depth, subsample, colsample_bytree))
    params = {
        "objective": "multi:softprob",
        "num_class": 12,
        "booster" : "gbtree",
        "eval_metric": "mlogloss",
        "eta": eta,
        "max_depth": max_depth,
        "subsample": subsample,
        "colsample_bytree": colsample_bytree,
        "silent": 1,
        "seed": random_state,
    }
    num_boost_round = 2000
    early_stopping_rounds = 100
    test_size = 0.3

    X_train, X_valid = train_test_split(train, test_size=test_size, random_state=random_state)
    print('Length train:', len(X_train.index))
    print('Length valid:', len(X_valid.index))
    y_train = X_train[target]
    y_valid = X_valid[target]
    dtrain = xgb.DMatrix(X_train[features], y_train)
    dvalid = xgb.DMatrix(X_valid[features], y_valid)

    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist, early_stopping_rounds=early_stopping_rounds, verbose_eval=True)

    print("Validating...")
    check = gbm.predict(xgb.DMatrix(X_valid[features]), ntree_limit=gbm.best_iteration)
    score = log_loss(y_valid.tolist(), check)

    print("Predict test set...")
    test_prediction = gbm.predict(xgb.DMatrix(test[features]), ntree_limit=gbm.best_iteration)

    print('Training time: {} minutes'.format(round((time.time() - start_time)/60, 2)))
    return test_prediction.tolist(), score

In [ ]:
def create_submission(score, test, prediction):
    # Make Submission
    now = datetime.datetime.now()
    sub_file = 'submission_' + str(score) + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
    print('Writing submission: ', sub_file)
    f = open(sub_file, 'w')
    f.write('device_id,F23-,F24-26,F27-28,F29-32,F33-42,F43+,M22-,M23-26,M27-28,M29-31,M32-38,M39+\n')
    total = 0
    test_val = test['device_id'].values
    for i in range(len(test_val)):
        str1 = str(test_val[i])
        for j in range(12):
            str1 += ',' + str(prediction[i][j])
        str1 += '\n'
        total += 1
        f.write(str1)
    f.close()

In [ ]:
#res, score = run_xgb(result_train, result_test, result_train.columns.values[:len(result_train.columns)-1], 'group', eta=0.01, random_state=0)
#print score
#create_submission(score, result_test, res)

In [ ]:
#create_submission(score, result_test, res)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import sklearn.cross_validation #импортируем кросс-валидацию

In [ ]:
sc = StandardScaler()
scaled_train = sc.fit_transform(result_train[result_train.columns.difference(['group','device_id'])])
scaled_test = sc.fit_transform(result_test[result_test.columns.difference(['group'])])

In [ ]:
import time
import datetime

bag_score = []#список результатов
c_est = [10**-2,10**-1,1,10,100,1000]#значения C для перебора
lr_models = []
for c in c_est:
    start_time = datetime.datetime.now()
    logr = LogisticRegression(penalty='l2',C=c,tol=0.000001,verbose=True)#логистическая регрессия
    #lr_models.append(logr.fit(scaled_train_bag, features_target))#обучаем класификатор
    bag_score.append(sklearn.cross_validation.cross_val_score(logr, scaled_train, result_train['group'].values,cv=3, scoring='log_loss').mean())
    #к списку результатов добавляем средний получившийся на кросс-валидации результат по roc_auc score
    print bag_score
    print 'Time elapsed for c =', c,':', datetime.datetime.now() - start_time

In [ ]:
print bag_score